In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
mew_url = 'https://bulbapedia.bulbagarden.net/wiki/151_(TCG)'
mew_page = requests.get(mew_url)

In [3]:
mew_soup = BeautifulSoup(mew_page.text, 'html')

In [4]:
# Incidentally, there is another table with the same formatting.
mew_table = mew_soup.find_all('table', class_ = 'roundy', style = 'float:left; background:#FF1111; border: 3px solid #000')[1]

In [6]:
#Creating Table Headers by specifying the top row.
mew_headers = mew_table.find('tr', style = 'background:#6464FF;')

In [7]:
#Strip the HTML tags for a List of column titles. Removing unnecessary column titles for this project.
mew_column_titles = [header.text.strip() for header in mew_headers.find_all('th')]
to_remove = ['Type', 'Mark', 'Promotion']
for title in to_remove:
    while title in mew_column_titles:
        mew_column_titles.remove(title)

In [9]:
#Create the dataframe with columns set.
mew_df = pd.DataFrame(columns = mew_column_titles)

In [10]:
#Incidentally, the type of the card is tagged as th, not td. It's unneeded for the purposes of this dataframe however.
mew_column_data = mew_table.find_all('td', style = 'background:#FFFFFF')

In [11]:
#Populate dataframe with column data. {'1': 'No.', '2': Mark, '3': 'Card name', '4': 'Rarity'}
column_number = 0
current_entry = []

for value in mew_column_data:
    row_to_update = len(mew_df)
    column_number += 1

    if column_number == 1:
        current_entry.append(value.text.strip())

    #Column 2, the Mark, is unnecessary so it's skipped.

    elif column_number == 3:
        #The ex (and other tags such as VStar) will not show up without additional code.
        card_name = value.text.strip()

        try:
            pokemon_tag = value.find_all('a')[1].get('title')
            card_name += ' ' + pokemon_tag
            current_entry.append(card_name)
        except:
            current_entry.append(card_name)

    elif column_number == 4:
        current_entry.append(value.find('a').get('title'))
        mew_df.loc[row_to_update] = current_entry

        #Reset for the next row.
        current_entry = []
        column_number = 0

In [13]:
mew_df.to_csv(r'C:/Users/zsere/Documents/Raw Data/Pokemon Set Card Info/151 Card Info.csv')